# Quickstart LSST Verify Demo

<br>Owner: **Keith Bechtol** ([@bechtol](https://github.com/LSSTScienceCollaborations/StackClub/issues/new?body=@bechtol))
<br>Minor updates by:   Douglas Tucker ([@douglasleetucker](https://github.com/LSSTScienceCollaborations/StackClub/issues/new?body=@douglasleetucker))
<br>Last Verified to Run: **v21.0.0**
<br>Verified Stack Release: **v21.00**, **v17.0**, **d_2019_02_04**, **d_2019_01_15**

This notebook demonstrates basic functionality of the LSST Verify python package: https://github.com/lsst/verify . The notebook is based on the documentation at https://sqr-019.lsst.io/ . 

Another example from the LSST Systems Engineering team can be found [here](https://github.com/mareuter/notebooks/blob/master/LSST/Systems_Engineering/System_Verification_SQuaSH/System_Verification_Demo.ipynb), for which the metrics are defined [here](https://github.com/mareuter/notebooks/tree/master/LSST/Systems_Engineering/System_Verification_SQuaSH).

### Learning Objectives

After working through and studying this notebook you should be able to
   1. Create custom metrics for your science cases and associate multiple specifications with those metrics
   2. Evaluate metrics and store the output for subsequent analysis
   3. Create customized summary displays for the performance on those metrics relative to your specifications

### Logistics

This notebook is intended to be runnable on `lsst-lsp-stable.ncsa.illinois.edu` from a local git clone of https://github.com/LSSTScienceCollaborations/StackClub.

## Setup

In [ ]:
# What version of the Stack am I using?
! echo $HOSTNAME
! eups list -s lsst_distrib

In [ ]:
import json
import numpy as np
from matplotlib import pyplot as plt
%matplotlib inline

import astropy.units as u

import lsst.verify

## Defining metrics and specifications

There are specific rules for the directory structure and naming. In particular, there are required folders for "metrics" and "specs". The metrics are defined in a set of yaml files in the metrics folder. For each yaml file of metrics, there is a corresponding directory with the same name in the specifications directory. The specifications are defined in their own yaml files. An example directory structure appears below.

In [ ]:
!tree verify_demo

Let's take a look at both the metric and specification yaml files

In [ ]:
!cat verify_demo/metrics/demo_astrometry.yaml

In [ ]:
!cat verify_demo/specs/demo_astrometry/specs.yaml

In the definition of specifications, note that the "---" line between specifications is required.

Next, we create instances of the `MetricSet` and `SpecificationSet`.

In [ ]:
METRIC_PACKAGE = "verify_demo"
metrics = lsst.verify.MetricSet.load_metrics_package(METRIC_PACKAGE)
specs = lsst.verify.SpecificationSet.load_metrics_package(METRIC_PACKAGE)

View the metrics that have been defined:

In [ ]:
metrics

View the specifications that have been defined:

In [ ]:
specs

## Computing and storing metrics

For the purpose of illustration, let's make up some measurement values corresponding to our metrics. The following lines are placeholders for the analysis that we would want to do. In this example, we choose measurement values that are intermediate between the specifications defined above so that we can see what happens when some specifications are met and others are not. Notice that the measurements can have dimensions.

In [ ]:
zp_rms = 15.*u.mmag
zp_meas = lsst.verify.Measurement('demo_photometry.ZeropointRMS', zp_rms)

astro_rms = 15.*u.mas
astro_meas = lsst.verify.Measurement('demo_astrometry.AstrometricRMS', astro_rms)

It is possible to include extra information along with the measurements. These are made up values only for the purpose of illustration.

In [ ]:
zp_meas.extras['x'] = lsst.verify.Datum(np.random.random(10) * u.mag, label="x", description="x-values")
zp_meas.extras['y'] = lsst.verify.Datum(np.random.random(10) * u.mag, label="y", description="y-values")

Create an LSST verify job and add the measurements.

In [ ]:
job = lsst.verify.Job(metrics=metrics, specs=specs)
job.measurements.insert(zp_meas)
job.measurements.insert(astro_meas)

Provide metadata about the job. This could be used to capture information about the analysis configuration, software version, dataset, etc.

In [ ]:
job.meta.update({'version': 'test'})

When we are done, write the output to a file. This can be exported to metric aggregators at a later time.

In [ ]:
job.write('demo.json')

## Creating reports

Create a report to visualize the outcome of our analysis. We already have the job in memory, but for the purpose of illustration, let's read in the file that we just wrote to show how one could examine the results at a later time.

In [ ]:
with open('demo.json') as f:
    job = lsst.verify.Job.deserialize(**json.load(f))

Display a summary report

In [ ]:
job.report().show()

Notice that because of the measurement values we used in this example, some of the specifications are met, while others are not.

It is possible to select particular tags to customize the report. The example below shows a selection on specification tags.

In [ ]:
job.report(spec_tags=['minimum']).show()

It is also possible to see what tags are available.

In [ ]:
job.metrics['demo_astrometry.AstrometricRMS'].tags

View metadata.

In [ ]:
job.meta

A lot of information is available for plotting if we want to dig deeper into the results. These are the extra data that we saved together with the metric values. 

In [ ]:
m = job.measurements['demo_photometry.ZeropointRMS']

plt.figure()
plt.scatter(m.extras['x'].quantity, m.extras['y'].quantity)
plt.xlabel('%s (%s)'%(m.extras['x'].label, m.extras['x'].unit.name))
plt.ylabel('%s (%s)'%(m.extras['y'].label, m.extras['y'].unit.name))
plt.title('%s; %s'%(m.metric_name.metric, job.meta["version"]))
plt.xlim(0, 1)
plt.ylim(0, 1)

Again, the particular values used in this example are just for demonstration purposes.